In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import glob

import numpy as np
import pandas as pd
from skimage import io

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from skimage import io
import matplotlib.pyplot as plt


import segmentation_models as sm
sm.set_framework('keras')

import keras
keras.backend.set_image_data_format('channels_last')

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from sklearn.model_selection import train_test_split

import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapOnImage

from keras.utils import to_categorical

# Valid
import time
import gc

import keras.backend as K

In [ ]:
LABELS = ['Branching', 'Fish', 'Massive', 'Not Massive', 'Substrate', 'Target', 'Water']

labels = {'Branching' : 0, 
          'Fish' : 1, 
          'Massive' : 2,
          'Not Massive' : 3,
          'Substrate' : 4,
          'Target' : 5,
          'Water' : 6}

NO_LABEL = 255


cp = np.array([ [178,24,43],
                [239,138,98],
                [253,219,199],
                [247,247,247],
                [209,229,240],
                [103,169,207],
                [33,102,172] ])

In [ ]:
path = "Z:\\Photogrammetry_data\\Florida\\7_16_2019_patch\\models\\before-algae\\Deep Learning\\data\\"

images = sorted(glob.glob(path + "images\\*.png"))
masks = sorted(glob.glob(path + "dense\\*.png"))
points = sorted(glob.glob(path + "sparse\\*.csv"))

data = pd.DataFrame(list(zip(images, masks, points)), columns = ['Images', 'Masks', 'Points'])

In [ ]:
train, valid = train_test_split(data, test_size = .1)

train.reset_index(drop = True, inplace = True)
valid.reset_index(drop = True, inplace = True)

len(train), len(valid)

In [ ]:
# Original 2160, 3840
# Must be divisible by 32(?)
height, width = 736, 1280 

In [ ]:
def colorize_prediction(pred):
   
    colored_mask = np.zeros(shape = (pred.shape[0], pred.shape[1], 3))

    for _ in np.unique(pred):
           
            colored_mask[pred == _] = cp[_]/255.0
        
    return colored_mask

# Augmentation methods
augs_for_images = iaa.Sequential([iaa.Resize(size = {'height' : height, 'width' : width}, interpolation = 'linear',
                                            random_state = 5),
                                  iaa.Fliplr(0.25, random_state = 1),
                                  iaa.Flipud(0.25, random_state = 2),
                                  iaa.Rot90([1, 2, 3, 4], True, random_state = 3)
                       ])


augs_for_masks = iaa.Sequential([iaa.Resize(size = {'height' : height, 'width' : width}, interpolation = 'nearest',
                                           random_state = 5),
                                  iaa.Fliplr(0.25, random_state = 1),
                                  iaa.Flipud(0.25, random_state = 2),
                                  iaa.Rot90([1, 2, 3, 4], True, random_state = 3)
                                ])



resize_for_images = iaa.Sequential([
     iaa.Resize(size = {'height' : height, 'width' : width}, interpolation = 'linear', random_state = 1),
])

resize_for_masks = iaa.Sequential([
     iaa.Resize(size = {'height' : height, 'width' : width}, interpolation = 'nearest', random_state = 1),
])


# Image data generator class
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, dataframe, batch_size, augment, n_classes = 8):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.augment = augment
          
        
    # Steps per epoch    
    def __len__(self):
        return len(self.dataframe) // self.batch_size
    
    # Shuffles and resets the index at the end of training epoch
    def on_epoch_end(self):
        self.dataframe = self.dataframe.reset_index(drop = True)
    
    
    # Generates data, feeds to training
    def __getitem__(self, index):
        
        global preprocess_input
        
        processed_images = []
        processed_masks = []
        
        for _ in range(self.batch_size):

            the_image = io.imread(self.dataframe['Images'][index])
            the_mask = io.imread(self.dataframe['Masks'][index]).astype('uint8');
            one_hot_mask = to_categorical(the_mask, 7)
            
            if(self.augment):
                
                processed_image = augs_for_images(image = the_image)
                processed_mask = augs_for_masks(image = one_hot_mask)
         
            else:
                # Still resizing and then random cropping, but no augmentations   
                processed_image = resize_for_images(image = the_image)
                processed_mask = resize_for_masks(image = one_hot_mask)

            processed_images.append(preprocess_input(processed_image))
            processed_masks.append(processed_mask)

                
        batch_x = np.array( processed_images )
        batch_y = np.array( processed_masks )
        
        return (batch_x, batch_y)


In [ ]:
# Parameters for training      
batch_size = 1
num_epochs = 100

train_steps = len(train) // batch_size; print(train_steps)
valid_steps = len(valid) // batch_size; print(valid_steps)

train_gen = DataGenerator(train, batch_size = batch_size, augment = True) 
valid_gen = DataGenerator(valid, batch_size = batch_size, augment = False)

In [ ]:
# View image and mask 
r = np.random.randint(len(data))

print(data.Images[r].split("\\")[-1])

the_image = io.imread(data['Images'][r])
the_mask = io.imread(data['Masks'][r])
the_cpce = pd.read_csv(data['Points'][r])

colors = [cp[labels[label]]/255.0 for label in the_cpce.Labels.values]

plt.figure(figsize = (20, 15))
plt.subplot(1, 3, 1)
plt.title("Image")
plt.imshow(the_image)

plt.subplot(1, 3, 2)
plt.title("Sparse Points")
plt.imshow(the_image)
plt.scatter(the_cpce.X.values, the_cpce.Y.values, s = 30, c = colors)

plt.subplot(1, 3, 3)
plt.title("Mask (SLIC)")
plt.imshow(colorize_prediction(the_mask), alpha = .85)
plt.show()

In [ ]:
BACKBONE = 'efficientnetb0'

preprocess_input = sm.get_preprocessing(BACKBONE) 

model = sm.Unet(input_shape = (None, None, 3), 
                backbone_name = BACKBONE, 
                encoder_weights = 'noisy-student', # 'imagenet'
                activation = 'softmax', 
                classes = 7,
                encoder_freeze = True,
                decoder_use_batchnorm = True)

In [ ]:
from segmentation_models.metrics import precision, recall, iou_score
from segmentation_models.losses import cce_jaccard_loss
from keras.optimizers import Adam

metrics = ['accuracy', iou_score, precision, recall]

model.compile(optimizer = Adam(lr = .001), 
              loss = [cce_jaccard_loss], 
              metrics = metrics)

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

holla = [
         ReduceLROnPlateau(monitor = 'val_loss', factor = .65, patience = 2, verbose = 1),
         
         ModelCheckpoint(filepath = 'weights\\SS\\Florida_' + BACKBONE + '_.h5', 
                         monitor = 'val_loss', save_weights_only = True, 
                         save_best_only = True, verbose = 1)
        ]

In [ ]:
history = model.fit_generator(generator = train_gen, 
                              steps_per_epoch = train_steps, 
                              epochs = num_epochs, 
                              validation_data = valid_gen,
                              validation_steps = valid_steps,
                              verbose = 1,
                              callbacks = holla)


In [ ]:
print(history.history.keys())

plt.figure(figsize= (10, 5))

plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(np.argmin(history.history["val_loss"]), 
         np.min(history.history["val_loss"]), 
         marker = "x", color = "b", label = "best model")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()

plt.figure(figsize= (10, 5))
plt.plot(history.history["precision"], label="precision")
plt.plot(history.history["val_precision"], label="val_precision")
plt.title("Training Precision")
plt.xlabel("Epoch #")
plt.ylabel("Precision")
plt.legend(loc="upper right")
plt.show()

plt.figure(figsize= (10, 5))
plt.plot(history.history["recall"], label="recall")
plt.plot(history.history["val_recall"], label="val_recall")
plt.title("Training Recall")
plt.xlabel("Epoch #")
plt.ylabel("Recall")
plt.legend(loc="upper right")

plt.show()

In [ ]:
model.load_weights("weights\\SS\\Florida_" + BACKBONE + '_.h5')

In [ ]:
# Changing the size of the test image provides slightly different results
# find the dimension(s) that provide the best testing-time augmentation for you
# This only works if the model was initialized with input shape (None, None, 3)

# 736,  1280 - .35
# 1088, 1920 - .5
# 1408, 2496 - .65
# 1632, 2880 - .75
# 1824, 3200 - .85
# 1984, 3680 - .92
# 2144, 3840 - .99


h = [1088, 1408, 1632, 1824]
w = [1920, 2496, 2880, 3200]

In [ ]:
def get_random_sample(dataframe):
    
    index = np.random.randint(len(dataframe))

    image = io.imread(dataframe['Images'][index])
    mask = io.imread(dataframe['Masks'][index])
    
    return image, mask, index


def mode(a, axis = 0):

    a = np.array(a)
    scores = np.unique(np.ravel(a))  # all unique values
    testshape = list(a.shape)        # dimensions of array
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape, dtype = int)    
    oldcounts = np.zeros(testshape, dtype = int)       

    for score in scores:

        template = (a == score)                                 
        counts = np.expand_dims(np.sum(template, axis), axis)

        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)  
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent

    return mostfrequent[0]


def make_prediction(image):
    
    masks = []
    predictions = []
    
    if(True):
        
        for _ in range(3):
    
            resized_image = iaa.Resize({'height' : h[_], 'width' : w[_]}).augment_image(image = image)

            y_pred = model.predict(np.expand_dims(preprocess_input(resized_image), axis = 0)).squeeze()
            
            y_pred = iaa.Resize({'height' : image.shape[0], 
                                 'width' : image.shape[1]}, 
                                      interpolation = 'nearest').augment_image(image = y_pred)

            predictions.append(y_pred)
            
            mask = np.argmax(y_pred, axis = 2).astype("uint8"); 
            
            masks.append(mask)

    return mode(np.array(masks)), predictions


def compare_results(image, mask, y_pred):

    plt.figure(figsize = (20, 15))

    plt.subplot(1, 3, 1)
    plt.title("Image")
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.title("Prediction")  
    plt.imshow(colorize_prediction(y_pred))
    
    plt.subplot(1, 3, 3)
    plt.title("Overlaid")
    plt.imshow(image)
    plt.imshow(colorize_prediction(y_pred), alpha = .65)
    
    plt.show()

In [ ]:

# Getting the predictions and comparing them
for _ in range(1):
    
    image, mask, index = get_random_sample(data)
    y_pred, predictions = make_prediction(image)

    compare_results(image, mask, y_pred)
    gc.collect()

In [ ]:
# Set a threshold to view only the predictions that model is confident about
# black out the regions it isn't.
threshold = .65

filtered_masks = []

for prediction in predictions:

    first = np.sort(prediction)[:, :, -1]
    second = np.sort(prediction)[:, :, -2]

    diff = first - second
    confidence = diff < threshold

    filtered_mask = np.argmax(prediction, axis = 2)
    filtered_mask[confidence] = 7
    
    filtered_masks.append(filtered_mask)
    
filtered_masks = mode(np.array(filtered_masks))

plt.figure(figsize = (20, 20))
plt.imshow(image)
plt.imshow(colorize_prediction(filtered_mask), alpha = .75)
plt.show()